In [1]:
import pandas as pd
from numpy import *
from datetime import datetime
from tabulate import tabulate
from bsmiv import OptionInputs, BlackScholesOptionPricing

In [13]:
def newton_iv(spot, strike, rate, dte, callprice=None, putprice=None):
    x0 = 1.
    h = 0.01
    tolerenace = 1e-7
    epsilon = 1e-14
    maxiter = 200


    if callprice:
        f = lambda x: BlackScholesOptionPricing(OptionInputs(
            spot=spot, 
            strike=strike, 
            rate=rate, 
            ttm=dte, 
            volatility=x
        )
    ).call_price - callprice
        
    if putprice:
        f = lambda x: BlackScholesOptionPricing(OptionInputs(
            spot=spot, 
            strike=strike, 
            rate=rate, 
            ttm=dte, 
            volatility=x
        )
    ).put_price - putprice

    for i in range (maxiter):
        y = f(x0)
        yprime = (f(x0+h) - f(x0-h))/(2*h)

        if abs(yprime)<epsilon:
            break

        x1 = x0 - y/yprime

        if (abs(x1 - x0)) <= tolerenace*abs(x1):
            break

        x0 = x1

    return(x1)


In [14]:
newton_iv(100,100,0.05,1,callprice=8)

0.13377582589298717

In [17]:
newton_iv(100,100,0.05,1,putprice=10)

0.3170185859464639

In [12]:
#to veryfy the output, the price of the call option calculated by BSM should be close to 8

BlackScholesOptionPricing(OptionInputs(spot = 100, strike=100, rate = 0.05, ttm = 1, volatility=0.13377582589298717)).call_price

8.000000000124928

In [19]:
BlackScholesOptionPricing(OptionInputs(spot = 100, strike=100, rate = 0.05, ttm = 1, volatility=0.3170185859464639)).put_price

10.000000000000384

## Bisection Method

The bisection method is considered to be one of the simplest and robust root finding algorithm. 

Suppose, we know the two points of an interval $a$ and $b$, where $a < b$ and $f(a)<0$ and $f(b)>0$ ;ie along the continuous function and the midpoint of this interval is $\frac{a+b}{2}$, then we can evaluate the value as $f(c)$
Iteratively, we replace $c$ as either $ a $ or $ b $, thereby shortening the interval to find the root. If $ f(c) = 0 $ or within an acceptable value, we have a root. Bisection methods are stable and guarantee to converge. As it does not require knowledge of the derivative, it takes more computational time.

In [37]:
def bisection_iv(spot, strike, rate, dte, callprice, putprice, b = 2.0, a = 0.0):

    tolerance = 1e-7

    #this is market price
    price = callprice if callprice else putprice

    for _ in range (1000):
        mid = (b+a)/2

        if mid < tolerance:
            break


        if callprice:
            estimate = BlackScholesOptionPricing(
                OptionInputs(
                    spot=spot, 
                    strike=strike, 
                    rate=rate, 
                    ttm=dte, 
                    volatility=mid
                )
            ).call_price
            
        if putprice:
            estimate = BlackScholesOptionPricing(
                OptionInputs(
                    spot=spot, 
                    strike=strike, 
                    rate=rate, 
                    ttm=dte, 
                    volatility=mid
                )
            ).put_price
            
        if round(estimate,7) == price:
            break
        elif estimate > price:
            b=mid
        elif estimate < price:
            a=mid

    return mid
            

    
            
        

In [38]:
bisection_iv(100,100,0.05,1,callprice=8., putprice=None)

0.13377582654356956

In [41]:
bisection_iv(100,100,0.05,1,putprice=10., callprice=None)

0.3170185871422291

In [42]:
BlackScholesOptionPricing(OptionInputs(spot=100, strike=100, rate=0.05, ttm=1, volatility=0.13377582588953527)).call_price

7.9999999999999645

In [43]:
BlackScholesOptionPricing(OptionInputs(spot=100, strike=100, rate=0.05, ttm=1, volatility=0.3170185871422291)).put_price

10.000000045377952

In [52]:
option = BlackScholesOptionPricing(OptionInputs(spot=100, strike = 100, rate=0.05, ttm = 1, volatility=0.10, callprice=8.))

header = ['option price', 'Delta', 'Gamma', 'Theta', 'Vega', 'rho', 'IV']
table = [[option.call_price, option.call_delta, option.gamma, option.call_theta, option.vega,option.call_rho, option.impvol]]

print(tabulate(table,header))

  option price    Delta      Gamma       Theta      Vega       rho        IV
--------------  -------  ---------  ----------  --------  --------  --------
       6.80496  0.70884  0.0342944  -0.0134758  0.342944  0.640791  0.133776


In [53]:
option = BlackScholesOptionPricing(OptionInputs(spot=100, strike = 100, rate=0.05, ttm = 1, volatility=0.10, putprice=10.))

header = ['option price', 'Delta', 'Gamma', 'Theta', 'Vega', 'rho', 'IV']
table = [[option.put_price, option.put_delta, option.gamma, option.put_theta, option.vega,option.put_rho, option.impvol]]

print(tabulate(table,header))

  option price     Delta      Gamma         Theta      Vega        rho        IV
--------------  --------  ---------  ------------  --------  ---------  --------
        1.9279  -0.29116  0.0342944  -0.000445276  0.342944  -0.310439  0.317019
